In [1]:
%reload_ext dotenv
%dotenv

In [2]:
%env LOGLEVEL=INFO

%load_ext autoreload
%autoreload 2

env: LOGLEVEL=INFO


In [3]:
import os, logging
import json

print(os.getenv("LOGLEVEL"))
# logging.basicConfig(level=os.environ.get("LOGLEVEL", "WARNING").upper())
log = logging.getLogger("portfolio-transactions")
log.setLevel(os.environ.get("LOGLEVEL", "WARNING").upper())

INFO


In [4]:
import sys
print(sys.path)

['/home/harjeet/playground/KiteConnect/option-charts/notebooks', '/home/harjeet/.pyenv/versions/3.8.3/lib/python38.zip', '/home/harjeet/.pyenv/versions/3.8.3/lib/python3.8', '/home/harjeet/.pyenv/versions/3.8.3/lib/python3.8/lib-dynload', '', '/home/harjeet/.virtualenvs/kiteconnect/lib/python3.8/site-packages', '/home/harjeet/playground/KiteConnect/option-charts/src', '/home/harjeet/.virtualenvs/kiteconnect/lib/python3.8/site-packages/IPython/extensions', '/home/harjeet/.ipython']


In [5]:
from kiteconnect import KiteConnect

In [6]:
API_KEY      = os.getenv("API_KEY")
API_SECRET   = os.getenv("API_SECRET")
ACCESS_TOKEN = os.getenv("ACCESS_TOKEN")

## Getting Access Token

Source: https://kite.zerodha.com/connect/login?v=3&api_key=xxx

In [7]:
!echo "https://kite.zerodha.com/connect/login?v=3&api_key=$API_KEY"

https://kite.zerodha.com/connect/login?v=3&api_key=hbviklv8pv4mx2uo


In [8]:
request_token = "Muhme8zqK83fWQfG82j3ucCP7Sin79YZ"

def get_access_token():
    kite = KiteConnect(api_key=os.getenv("API_KEY")) # api_key is in above url
    data = kite.generate_session(request_token, os.getenv("API_SECRET"))
    access_token = data["access_token"]
    return access_token

# get_access_token()

In [9]:
!pip list

Package             Version                 Location
------------------- ----------------------- ------------------------------------------------------
argon2-cffi         20.1.0
astroid             2.4.2
async-generator     1.10
attrs               20.2.0
autobahn            20.7.1
Automat             20.2.0
backcall            0.2.0
bleach              3.2.1
certifi             2020.6.20
cffi                1.14.3
chardet             3.0.4
constantly          15.1.0
cryptography        3.2.1
decorator           4.4.2
defusedxml          0.6.0
entrypoints         0.3
enum34              1.1.10
hyperlink           20.0.1
idna                2.10
incremental         17.5.0
ipykernel           5.3.4
ipython             7.19.0
ipython-genutils    0.2.0
isort               5.6.4
jedi                0.17.2
Jinja2              2.11.2
json5               0.9.5
jsonschema          3.2.0
jupyter-client      6.1.7
jupyter-core        4.6.3
jupyterlab          2.2.9
jupyterlab-pygments 0.1.2
jupy

## Get Tick

In [10]:
instruments = [12219650, 12219906]

In [11]:
import logging
from kiteconnect import KiteTicker

logging.basicConfig(level=logging.DEBUG)

# Initialise
# kws = KiteTicker("your_api_key", "your_access_token")
kws = KiteTicker(API_KEY, ACCESS_TOKEN)

def on_ticks(ws, ticks):
    # Callback to receive ticks.
    
    # on json.dumps
    # TypeError: Object of type datetime is not JSON serializable formatting issue on datetime object 
    #     http://127.0.0.1:7777/?token=473bbc2c11c1b9b0865b35b31c0ba704c151a06b833abce7
    logging.debug("Ticks: {}".format(json.dumps(ticks, indent=4, default=str)))

def on_connect(ws, response):
    # Callback on successful connect.
    # Subscribe to a list of instrument_tokens (RELIANCE and ACC here).
#     ws.subscribe([738561, 5633])
    ws.subscribe(instruments)

    # Set RELIANCE to tick in `full` mode.
#     ws.set_mode(ws.MODE_FULL, [738561])
    ws.set_mode(ws.MODE_FULL, instruments)

def on_close(ws, code, reason):
    # On connection close stop the main loop
    # Reconnection will not happen after executing `ws.stop()`
    ws.stop()

# Assign the callbacks.
kws.on_ticks = on_ticks
kws.on_connect = on_connect
kws.on_close = on_close


```json
DEBUG:root:Ticks: [{'tradable': True, 'mode': 'quote', 'instrument_token': 12219650, 'last_price': 212.9, 'last_quantity': 75, 'average_price': 227.14, 'volume': 1348650, 'buy_quantity': 61500, 'sell_quantity': 67575, 'ohlc': {'open': 265.1, 'high': 268.3, 'low': 205.05, 'close': 264.15}, 'change': -19.401855006625016}, {'tradable': True, 'mode': 'quote', 'instrument_token': 12219906, 'last_price': 272.65, 'last_quantity': 75, 'average_price': 267.43, 'volume': 92700, 'buy_quantity': 58350, 'sell_quantity': 45750, 'ohlc': {'open': 269.6, 'high': 294.15, 'low': 235.05, 'close': 262.5}, 'change': 3.866666666666658}]
DEBUG:root:Ticks: [{'tradable': True, 'mode': 'quote', 'instrument_token': 12219906, 'last_price': 272.65, 'last_quantity': 75, 'average_price': 267.43, 'volume': 92700, 'buy_quantity': 57000, 'sell_quantity': 47025, 'ohlc': {'open': 269.6, 'high': 294.15, 'low': 235.05, 'close': 262.5}, 'change': 3.866666666666658}]
```


**ws.MODE_FULL**

```json
DEBUG:root:Ticks: [{'tradable': True, 'mode': 'full', 'instrument_token': 12219650, 'last_price': 210.55, 'last_quantity': 75, 'average_price': 227.19, 'volume': 1344000, 'buy_quantity': 61650, 'sell_quantity': 64725, 'ohlc': {'open': 265.1, 'high': 268.3, 'low': 205.05, 'close': 264.15}, 'change': -20.291501041075136, 'last_trade_time': datetime.datetime(2020, 11, 4, 12, 2, 46), 'oi': 834150, 'oi_day_high': 834150, 'oi_day_low': 735225, 'timestamp': datetime.datetime(2020, 11, 4, 12, 2, 50), 'depth': {'buy': [{'quantity': 150, 'price': 210.0, 'orders': 2}, {'quantity': 75, 'price': 209.9, 'orders': 1}, {'quantity': 75, 'price': 209.85, 'orders': 1}, {'quantity': 75, 'price': 209.8, 'orders': 1}, {'quantity': 75, 'price': 209.7, 'orders': 1}], 'sell': [{'quantity': 75, 'price': 210.45, 'orders': 1}, {'quantity': 75, 'price': 210.5, 'orders': 1}, {'quantity': 150, 'price': 210.6, 'orders': 2}, {'quantity': 300, 'price': 210.65, 'orders': 2}, {'quantity': 150, 'price': 210.7, 'orders': 2}]}}, {'tradable': True, 'mode': 'full', 'instrument_token': 12219906, 'last_price': 277.0, 'last_quantity': 75, 'average_price': 267.28, 'volume': 90825, 'buy_quantity': 53925, 'sell_quantity': 44625, 'ohlc': {'open': 269.6, 'high': 294.15, 'low': 235.05, 'close': 262.5}, 'change': 5.523809523809524, 'last_trade_time': datetime.datetime(2020, 11, 4, 12, 2, 49), 'oi': 54525, 'oi_day_high': 56250, 'oi_day_low': 50100, 'timestamp': datetime.datetime(2020, 11, 4, 12, 2, 51), 'depth': {'buy': [{'quantity': 75, 'price': 277.0, 'orders': 1}, {'quantity': 150, 'price': 276.05, 'orders': 1}, {'quantity': 225, 'price': 276.0, 'orders': 1}, {'quantity': 75, 'price': 275.9, 'orders': 1}, {'quantity': 600, 'price': 275.8, 'orders': 1}], 'sell': [{'quantity': 825, 'price': 277.9, 'orders': 2}, {'quantity': 75, 'price': 278.0, 'orders': 1}, {'quantity': 150, 'price': 278.1, 'orders': 1}, {'quantity': 1125, 'price': 278.15, 'orders': 1}, {'quantity': 1125, 'price': 278.25, 'orders': 1}]}}]
DEBUG:root:Ticks: [{'tradable': True, 'mode': 'full', 'instrument_token': 12219906, 'last_price': 277.0, 'last_quantity': 75, 'average_price': 267.28, 'volume': 90825, 'buy_quantity': 53925, 'sell_quantity': 44625, 'ohlc': {'open': 269.6, 'high': 294.15, 'low': 235.05, 'close': 262.5}, 'change': 5.523809523809524, 'last_trade_time': datetime.datetime(2020, 11, 4, 12, 2, 49), 'oi': 54525, 'oi_day_high': 56250, 'oi_day_low': 50100, 'timestamp': datetime.datetime(2020, 11, 4, 12, 2, 51), 'depth': {'buy': [{'quantity': 75, 'price': 277.0, 'orders': 1}, {'quantity': 150, 'price': 276.05, 'orders': 1}, {'quantity': 225, 'price': 276.0, 'orders': 1}, {'quantity': 75, 'price': 275.9, 'orders': 1}, {'quantity': 600, 'price': 275.8, 'orders': 1}], 'sell': [{'quantity': 825, 'price': 277.9, 'orders': 2}, {'quantity': 75, 'price': 278.0, 'orders': 1}, {'quantity': 150, 'price': 278.1, 'orders': 1}, {'quantity': 1125, 'price': 278.15, 'orders': 1}, {'quantity': 1125, 'price': 278.25, 'orders': 1}]}}, {'tradable': True, 'mode': 'full', 'instrument_token': 12219650, 'last_price': 210.55, 'last_quantity': 75, 'average_price': 227.19, 'volume': 1344000, 'buy_quantity': 60150, 'sell_quantity': 64800, 'ohlc': {'open': 265.1, 'high': 268.3, 'low': 205.05, 'close': 264.15}, 'change': -20.291501041075136, 'last_trade_time': datetime.datetime(2020, 11, 4, 12, 2, 46), 'oi': 834150, 'oi_day_high': 834150, 'oi_day_low': 735225, 'timestamp': datetime.datetime(2020, 11, 4, 12, 2, 51), 'depth': {'buy': [{'quantity': 150, 'price': 210.0, 'orders': 2}, {'quantity': 75, 'price': 209.9, 'orders': 1}, {'quantity': 75, 'price': 209.85, 'orders': 1}, {'quantity': 75, 'price': 209.8, 'orders': 1}, {'quantity': 75, 'price': 209.7, 'orders': 1}], 'sell': [{'quantity': 75, 'price': 210.3, 'orders': 1}, {'quantity': 75, 'price': 210.45, 'orders': 1}, {'quantity': 75, 'price': 210.5, 'orders': 1}, {'quantity': 150, 'price': 210.6, 'orders': 2}, {'quantity': 300, 'price': 210.65, 'orders': 2}]}}]
```


In [ ]:
kws.connect()